## 研究助手 MCP 流程图

https://excalidraw.com/#json=LCuFLuyEu6RVmI88gOWyo,9z6WL7N25Whlyfb2asTA2Q

### 一、结构说明

- **总体架构**：中央协调层 **研究助手 MCP（Orchestrator）** → 四类专家代理并行/串联协作 → 支持工具层（搜索、RAG、代码执行、可视化、文献管理等） → 输出层（文献包、数据分析报告、优化图表、最终稿）
- **信息流**：需求输入 → 任务分解（MCP）→ 分派给对应专家 → 各专家的输出回传给 MCP 进行聚合与反思 → 若需迭代由 MCP 发起反思/再分派
- **核心设计原则**：任务分解（task decomposition）、可插拔工具（Tools use）、组件级评估（component-level evals）、LLM规划（Planning）、反思与多轮迭代以改进输出

### 二、流程图关键节点（供绘图时使用的图形元素与连接说明）

- **节点**：MCP（矩形，置顶）；资料搜集、数据分析、图表优化、写作编辑（四个并列矩形）；工具总线（水平底层带状）；评估与反思模块（MCP 旁的循环节点）；外部系统（Zotero、数据库、API）为云图标。
- **连接**：MCP -> 专家（下行箭头）；专家 -> 工具总线（双向箭头）；专家 -> MCP（汇总箭头）；评估模块 -> MCP（循环箭头触发迭代）。
- **可视注记**：在每个专家节点下添加三行简短元数据：任务（Task）；核心能力/特征（Traits）；首选工具（Tools）。

### 三、流程图文本内容（每个框内要写的最小信息量）

- **MCP**：任务分解；调度与版本控制；跨代理一致性校验；最终合并与反思。
- **资料搜集专家**：任务：全面检索、快速摘录、生成参考条目；特征：真实性检查、来源可信度评分、文献因子（时间、引用、方法）检测；工具：web search, RAG, Zotero API, 文献矩阵导出。
- **数据分析专家**：任务：清洗、统计、建模、可复现分析脚本；特征：数据完整性校验、假设检验、可重现 notebook 输出；工具：Pandas/Polars, SQL, Jupyter/Colab, 可执行沙箱。
- **图表优化专家**：任务：图表设计、可读性与色彩优化、交互化产出；特征：视觉最佳实践、图说生成、可访问性检查；工具：Plotly/Altair/Matplotlib, Vega-Lite, D3 export。
- **写作编辑专家**：任务：结构草拟、风格统一、可发表稿件；特征：引用格式校验、语言润色、可重复性说明；工具：LLM 编辑模块、Grammarly/语法检查、参考文献格式化Reference formatter、引用检查Citation check。
- **评估与反思**：任务：组件级评估（自动化＋人工抽样）、反馈记录与策略调整；工具：Evals 框架、评分 rubrics、LLM 作为评审器。

## 搭建大纲（分阶段、可落地的实施计划）

### 第一阶段：需求与设计（2周）

1. 明确目标研究范畴与典型用例；定义成功指标（速度、准确率、可复现性）。
2. 设计代理角色与最小接口协议（任务输入格式、输出 schema、元数据字段）。
3. 选定核心工具栈（搜索/RAG、文献管理、数据处理、可视化、CI/执行沙箱）。
- 设计参考：任务分解与 component-level evals 思路.

### 第二阶段：最小可运行原型 MCP（4周）

1. 搭建 MCP 调度器（接收请求、任务分解、分派/收集结果、简单反思规则）。
2. 实现资料搜集代理（集成网页搜索 + Zotero 导入；输出：文献包 + evidence matrix）。
3. 实现数据分析代理（接收原始数据，返回清洗脚本 + 分析结果摘要 + 可复现 notebook）。
4. 简单图表与写作代理（图表模板 + 文本初稿生成）。
5. 端到端 demo：MCP 协调完成一次小规模研究任务并产出可审查结果。

### 第三阶段：工具链扩展与评估（4–6周）

1. 接入 RAG、知识库（向量 DB）、代码执行沙箱以支持复杂检索与可执行计划。
2. 建立组件级评估框架：自动化测试集 + LLM 评分 rubrics + 人工抽样复核.
3. 优化代理间的中间表示（JSON schema），确保可替换性与可审计性。
4. 加入版本控制与审计日志（结果可回溯）。

### 第四阶段：可用性与可视化（3周）

1. 构建交互界面（简单 web dashboard）：发起任务、查看进度、人工干预、下载产物。
2. 图表优化专家产出可交互图表，并支持导出 PNG/SVG/HTML。
3. 集成引用管理（Zotero）导出多种引用格式。

### 第五阶段：迭代与规模化（持续）

1. 根据评估数据调整代理策略、改进反思回路、增加更多专长代理（例如伦理审查、实验设计）。
2. 自动化流水线（CI）与安全审查（沙箱、权限控制、数据隐私）。
3. 编写使用手册、模板集合与评估报告生成器。

<aside>
💡

### 技术与实现建议（快速清单）

---

- 接口协议：用 JSON Schema 明确定义每类任务的 input/output，便于评估与替换。
- RAG 与向量库：短期先用 open-source 向量 DB；长期考虑私有化部署以保证数据控制。
- 可复现性：所有分析通过 notebook + Docker 化环境捕捉依赖版本。
- 评估：构建 ground-truth 数据集与 rubrics，把 LLM 用作第一轮评审并保留人工抽检流程.
- 数据安全：敏感数据在沙箱执行并加密存储，工具访问采用最小权限原则。
- 可替换性：各代理实现为独立微服务或函数，MCP 通过适配器接入。
</aside>

请继续下一步，请注意每次只做一步，并解释这一步的含义。

运行出现问题，这需要怎样解决？

好了，运行成功，参考项目逻辑框架和流程图，现在下一从哪里迭代？

### 测试

In [1]:
# 在 Notebook 的一个单元格里执行
import os
from dotenv import load_dotenv
load_dotenv()  # 加载 .env（包含 API Key、NOBEL_LLM_MODEL 等）

# 可选：临时覆盖主题与模型（如需）
# os.environ["NOBEL_THEME"] = "诺贝尔奖"
# os.environ["NOBEL_LLM_MODEL"] = "dashscope:qwen3-max"  # 或 "deepseek:deepseek-chat"

from src.mcp.orchestrator import run_nobel_pipeline

out_path = run_nobel_pipeline()
print("Run log ->", out_path)

Run complete. Log saved -> artifacts/nobel/run_log_1761723938.json (and artifacts/nobel/runs/1761723919/run_log.json)
Run log -> artifacts/nobel/run_log_1761723938.json
